In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

import GPyOpt

import argparse

from utils import tf_config, preprocess_data, search_algorithms, train, losses_utils, metrics

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

import time

dataset="01"
memory_limit=1500
n_individuals=10
interval_eeg=10

tf_config.set_seed(seed=42)
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

with tf.device('/CPU:0'):
    train_data, _ = preprocess_data.dataset(dataset, n_individuals=n_individuals,
                                            interval_eeg=interval_eeg, 
                                            ind_volume_fit=False,
                                            standardize_fmri=True,
                                            iqr=False,
                                            verbose=True)
    eeg_train, fmri_train =train_data

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy
I: Starting to Load Data
I: Finished Loading Data
I: Pairs Created


In [2]:
eeg_train = eeg_train[:100]
fmri_train = fmri_train[:100]

In [3]:
eeg_train.shape

(100, 64, 134, 10, 1)

## Build fMRI AE

In [4]:
theta = (0.002980911194116198, 0.0004396489214334123, (9, 9, 4), (1, 1, 1), 4, (7, 7, 7), 4, True, True, True, True, 3, 1)

In [5]:
#unroll hyperparameters
learning_rate=float(theta[0])
weight_decay = float(theta[1])
kernel_size = theta[2]
stride_size = theta[3]
batch_size=int(theta[4])
latent_dimension=theta[5]
n_channels=int(theta[6])
max_pool=bool(theta[7])
batch_norm=bool(theta[8])
skip_connections=bool(theta[9])
dropout=bool(theta[10])
n_stacks=int(theta[11])
outfilter=int(theta[12])
local=True

In [6]:
class EEG_to_fMRI(tf.keras.Model):
    
    def __init__(self, latent_shape, input_shape, kernel_size, stride_size, n_channels,
                maxpool=True, weight_decay=0.000, 
                skip_connections=False, batch_norm=True,
                n_stacks=2, local=True, 
                seed=None, fmri_args=None):
        super(EEG_to_fMRI, self).__init__()
        
        self.fmri_ae = fmri_ae.fMRI_AE(*fmri_args)
        
        self.build_encoder(latent_shape, input_shape, kernel_size, 
                            stride_size, n_channels, maxpool=maxpool,
                            weight_decay=weight_decay, skip_connections=skip_connections,
                            n_stacks=n_stacks, local=local, seed=seed)
        self.build_decoder()
        
    def build_encoder(self, latent_shape, input_shape, kernel_size, 
                            stride_size, n_channels, maxpool=True,
                            weight_decay=0.000, skip_connections=False,
                            n_stacks=2, local=True, seed=None):
        
        input_shape = tf.keras.layers.Input(shape=input_shape)
        
        x = input_shape
        previous_block_x = input_shape

        for i in range(n_stacks):
            x = fmri_ae.stack(x, previous_block_x, tf.keras.layers.Conv3D, 
                        kernel_size, stride_size, n_channels,
                        maxpool=maxpool, batch_norm=batch_norm, weight_decay=weight_decay, 
                        skip_connections=skip_connections, seed=seed)
            previous_block_x=x

        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.experimental.RandomFourierFeatures(latent_shape[0]*latent_shape[1]*latent_shape[2],
                                                              trainable=True)(x)
        
        if(dropout):
            x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Reshape(latent_shape)(x)
        
        self.eeg_encoder = tf.keras.Model(input_shape, x)
        self.fmri_encoder = self.fmri_ae.encoder
        
    def build_decoder(self):
        self.decoder = self.fmri_ae.decoder
        
    def build(self, input_shape1, input_shape2):
        self.eeg_encoder.build(input_shape=input_shape1)
        
        self.fmri_ae.build(input_shape=input_shape2)
        self.fmri_encoder.build(input_shape=input_shape2)
        
        model.built=True
    
    def call(self, X, training=True):
        x1, x2 = X
        
        z1 = self.eeg_encoder(x1)
        z2 = self.fmri_encoder(x2)
        
        if(training):
            return [self.decoder(z1), z1, z2]
        return self.decoder(z1)

In [9]:
import importlib
importlib.reload(fmri_ae)
importlib.reload(eeg_to_fmri)

with tf.device('/CPU:0'):
    model = eeg_to_fmri.EEG_to_fMRI(latent_dimension, eeg_train.shape[1:], (10,20,2), (1,1,1), 4,
                        maxpool=True, weight_decay=0.000, skip_connections=True,
                        batch_norm=True, dropout=False,
                        n_stacks=2, local=True, seed=None, 
                        fmri_args = (latent_dimension, fmri_train.shape[1:], 
                        kernel_size, stride_size, n_channels, 
                        max_pool, batch_norm, weight_decay, skip_connections,
                        n_stacks, True, False, outfilter, dropout))
    
    model.build(eeg_train.shape, fmri_train.shape)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = losses_utils.mse_cosine

    train_set = tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(batch_size)
    dev_set= tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(1)

<class 'tensorflow.python.keras.layers.convolutional.Conv3D'>
<class 'tensorflow.python.keras.layers.convolutional.Conv3D'>
<class 'tensorflow.python.keras.layers.convolutional.Conv3D'>
<class 'tensorflow.python.keras.layers.convolutional.Conv3D'>
<class 'tensorflow.python.keras.layers.convolutional.Conv3D'>


In [10]:
loss_history = train.train(train_set, model, optimizer, 
                            loss_fn, epochs=10, 
                            u_architecture=True,
                            val_set=dev_set, verbose=True, verbose_batch=True)[0]

Batch ... with loss: 1.501963
Batch ... with loss: 1.4254586
Batch ... with loss: 1.3694496
Batch ... with loss: 1.2983569
Batch ... with loss: 1.187983
Batch ... with loss: 1.0915617
Batch ... with loss: 1.0210397
Batch ... with loss: 0.97537935
Batch ... with loss: 0.9373429
Batch ... with loss: 0.9195987
Batch ... with loss: 0.8990369
Batch ... with loss: 0.885739
Batch ... with loss: 0.8679184
Batch ... with loss: 0.8594571
Batch ... with loss: 0.8520824
Batch ... with loss: 0.8466145
Batch ... with loss: 0.8533242
Batch ... with loss: 0.84611297
Batch ... with loss: 0.84913194
Batch ... with loss: 0.8487206
Batch ... with loss: 0.8530009
Batch ... with loss: 0.8499419
Batch ... with loss: 0.8462373
Batch ... with loss: 0.8489804
Batch ... with loss: 0.834661
Epoch 1 with loss: 0.9827637147903442
Epoch 1 with l2loss: 0.02161714982241392
Batch ... with loss: 0.8436123
Batch ... with loss: 0.83708155
Batch ... with loss: 0.8418976
Batch ... with loss: 0.8373609


KeyboardInterrupt: 

## Prediction vs ground truth plot

In [ ]:
from utils import viz_utils

save_path = "/home/david/eeg_to_fmri/plots/100_instances/"

instance = 1
for instance_x, instance_y in dev_set.repeat(1):
    fig = viz_utils.plot_3D_representation_projected_slices(instance_y.numpy()[0])
    plt.savefig(save_path + str(instance) + "_ground_truth.pdf", format="pdf")
    
    fig = viz_utils.plot_3D_representation_projected_slices(model([instance_x, instance_y])[0].numpy()[0])
    plt.savefig(save_path + str(instance) + "_predicted.pdf", format="pdf")
    
    instance += 1

## Structural Similarity Index

In [ ]:
metrics.ssim(dev_set, model)